<a href="https://colab.research.google.com/github/BhushanPawar-01/Intrusion_Detection_System/blob/main/IDS_Hybird.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Intrusion Detection System (A Hybrid Model)**

In [ ]:
# Install huggingface hub
!pip install huggingface-hub

In [ ]:
# Install datasets to import dataset direrctly from hugging face datasets
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00


In [ ]:
# Import import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

In [ ]:
# Load NSL-KDD dataset

from datasets import load_dataset
dataset = load_dataset("Mireu-Lab/NSL-KDD", download_mode="force_redownload")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
# Define train and test dataframe
train = pd.DataFrame(dataset['train'])
test = pd.DataFrame(dataset['test'])

In [ ]:
train.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal




---


**Data Preprocessing and Understanding**

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151165 entries, 0 to 151164
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     151165 non-null  int64  
 1   protocol_type                151165 non-null  object 
 2   service                      151165 non-null  object 
 3   flag                         151165 non-null  object 
 4   src_bytes                    151165 non-null  int64  
 5   dst_bytes                    151165 non-null  int64  
 6   land                         151165 non-null  int64  
 7   wrong_fragment               151165 non-null  int64  
 8   urgent                       151165 non-null  int64  
 9   hot                          151165 non-null  int64  
 10  num_failed_logins            151165 non-null  int64  
 11  logged_in                    151165 non-null  int64  
 12  num_compromised              151165 non-null  int64  
 13 

In [ ]:
train['protocol_type'].unique()

array(['tcp', 'udp', 'icmp'], dtype=object)

In [ ]:
train['service'].unique()

array(['ftp_data', 'other', 'private', 'http', 'remote_job', 'name',
       'netbios_ns', 'eco_i', 'mtp', 'telnet', 'finger', 'domain_u',
       'supdup', 'uucp_path', 'Z39_50', 'smtp', 'csnet_ns', 'uucp',
       'netbios_dgm', 'urp_i', 'auth', 'domain', 'ftp', 'bgp', 'ldap',
       'ecr_i', 'gopher', 'vmnet', 'systat', 'http_443', 'efs', 'whois',
       'imap4', 'iso_tsap', 'echo', 'klogin', 'link', 'sunrpc', 'login',
       'kshell', 'sql_net', 'time', 'hostnames', 'exec', 'ntp_u',
       'discard', 'nntp', 'courier', 'ctf', 'ssh', 'daytime', 'shell',
       'netstat', 'pop_3', 'nnsp', 'IRC', 'pop_2', 'printer', 'tim_i',
       'pm_dump', 'red_i', 'netbios_ssn', 'rje', 'X11', 'urh_i',
       'http_8001', 'aol', 'http_2784', 'tftp_u', 'harvest'], dtype=object)

In [ ]:
train['flag'].unique()

array(['SF', 'S0', 'REJ', 'RSTR', 'SH', 'RSTO', 'S1', 'RSTOS0', 'S3',
       'S2', 'OTH'], dtype=object)

In [ ]:
types = train['class'].unique()
types

array(['normal', 'anomaly'], dtype=object)

In [ ]:
fig = px.pie(train, values=train['class'].value_counts(), names=train['class'].unique(), title='class',width = 500, height = 500)
fig.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder

cols = ['protocol_type','service','flag','class']

for i in cols:
  en = LabelEncoder()
  train[i] = en.fit_transform(train[i])
  test[i] = en.fit_transform(test[i])

In [ ]:
train.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,1,20,9,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,1
1,0,2,44,9,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,1
2,0,1,49,5,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,0
3,0,1,24,9,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,1
4,0,1,24,9,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1


In [ ]:
test.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,13,1,55,9,118,2425,0,0,0,0,...,10,0.38,0.12,0.04,0.0,0.00,0.00,0.12,0.3,0
1,0,2,45,9,44,0,0,0,0,0,...,254,1.00,0.01,0.01,0.0,0.00,0.00,0.00,0.0,0
2,0,1,55,8,0,44,0,0,0,0,...,79,0.31,0.61,0.00,0.0,0.21,0.68,0.60,0.0,0
3,0,2,45,9,53,55,0,0,0,0,...,255,1.00,0.00,0.87,0.0,0.00,0.00,0.00,0.0,1
4,0,1,45,10,0,0,0,0,0,0,...,1,0.06,1.00,1.00,0.0,1.00,1.00,0.00,0.0,0


In [ ]:
# Create new dataframe
new_df = pd.concat([train, test])

In [ ]:
new_df.shape

(185559, 42)

In [ ]:
X,y = new_df.iloc[:,:-1], new_df.iloc[:,-1]

In [ ]:
print(X.shape)
print(y.shape)

(185559, 41)
(185559,)




---


**Model Building and Training**

In [ ]:
# Import libraries for building CNN architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution1D, MaxPooling1D, Flatten
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# CNN architecture for feature extraction
model = Sequential()
model.add(Convolution1D(41, kernel_size=41, activation="relu", input_shape=(41, 1), name="conv1d_1"))
model.add(MaxPooling1D(5,padding='same'))
model.add(Flatten())

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 1, 41)             1722      
                                                                 
 max_pooling1d (MaxPooling1  (None, 1, 41)             0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 41)                0         
                                                                 
Total params: 1722 (6.73 KB)
Trainable params: 1722 (6.73 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Extract features using the CNN model
fe = model.predict(X)
features = pd.DataFrame(fe)

5799/5799 [==============================] - 11s 2ms/step


In [ ]:
features.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,0.000000,0.000000,19.544703,27.669258,9.861340,0.0,26.046818,0.0,0.0,24.591331,...,20.508362,22.661118,33.699455,0.000000,6.008873,0.000000,6.995698,16.098431,0.00000,11.807006
1,10.764718,7.629187,15.141297,19.209404,12.589491,0.0,19.022738,0.0,0.0,3.827569,...,13.235326,3.585391,14.629416,1.816805,0.000000,9.917578,11.480715,0.000000,0.00000,11.589525
2,16.793726,14.678989,18.589521,13.450128,13.090851,0.0,12.489017,0.0,0.0,0.000000,...,5.394503,0.000000,1.446113,5.818542,0.000000,12.352612,3.328352,0.000000,0.00000,10.798501
3,436.549988,154.303268,10.735537,288.881866,0.000000,0.0,374.744202,0.0,0.0,411.402496,...,51.574440,215.038132,4.563942,0.000000,171.251266,0.000000,469.046600,0.000000,286.72406,64.907875
4,27.240772,15.651825,25.791315,35.815201,18.352039,0.0,39.070549,0.0,0.0,36.925472,...,31.356541,18.649782,30.581867,3.607639,2.758119,13.698469,24.272739,0.000000,0.00000,19.897213


In [ ]:
# Split freature dataframe and y in train and test data for RandomForest Model

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.30, random_state=42)


In [ ]:
rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(X_train, y_train)



---
**Model Performance Analysis on Test Data**


In [ ]:
pred= rf_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, pred)
print("Accuracy:", accuracy)


Accuracy: 0.9954911259610548


In [ ]:
from sklearn.metrics import classification_report

Classification_Report = classification_report(y_test, pred)
print('Classification Report is : \n', Classification_Report)

Classification Report is : 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00     27768
           1       1.00      0.99      1.00     27900

    accuracy                           1.00     55668
   macro avg       1.00      1.00      1.00     55668
weighted avg       1.00      1.00      1.00     55668

